In [141]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import pickle
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from stop_words import get_stop_words
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
from functools import lru_cache
pd.set_option('max_colwidth', 800)
pd.set_option('display.expand_frame_repr', False)

In [129]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [130]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


In [131]:
# dictionary for processing text
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [132]:
# for test and experiments 
test = "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked"
new_text = re.sub(r'@[\w]*', ' ', test)
new_text = new_text.lower()
new_text = [word for word in new_text.split() if len(word) > 1]
new_text = [apostrophe_dict.get(word) if word in apostrophe_dict else word for word in new_text]
new_text = [short_word_dict.get(word) if word in short_word_dict else word for word in new_text]
new_text = [emoticon_dict.get(word) if word in emoticon_dict else word for word in new_text]
new_text = [re.sub(r'[^\w\s]', '', word) for word in new_text]
new_text = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in new_text]
new_text = [re.sub(r'[^a-zA-Z]', '', word) for word in new_text]
new_text

['thanks',
 'for',
 'lyft',
 'credit',
 'cannot',
 'use',
 'cause',
 'they',
 'donot',
 'offer',
 'wheelchair',
 'vans',
 'in',
 'pdx',
 'disapointed',
 'getthanked']

In [133]:
def clean_text(text):
    new_text = re.sub(r'@[\w]*', ' ', text)
    new_text = new_text.lower()
    new_text = [word for word in new_text.split() if len(word) > 1]
    new_text = [apostrophe_dict.get(word) if word in apostrophe_dict else word for word in new_text]
    new_text = [short_word_dict.get(word) if word in short_word_dict else word for word in new_text]
    new_text = [emoticon_dict.get(word) if word in emoticon_dict else word for word in new_text]
    new_text = [re.sub(r'[^\w\s]', '', word) for word in new_text]
    new_text = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in new_text]
    new_text = [re.sub(r'[^a-zA-Z]', '', word) for word in new_text]
    return new_text

In [134]:
# create new data col 'tweet_token'
combine_df['tweet_token'] = combine_df.apply(lambda x: clean_text(x['tweet']), axis = 1)

In [135]:
def del_stopwords(text):
    return [word for word in text if not word in stopwords.words("english")]

In [136]:
# deleting stopword and create new col 'tweet_token_filtered'
combine_df['tweet_token_filtered'] = combine_df.apply(lambda x: del_stopwords(x['tweet_token']), axis = 1)

In [137]:
# apply stemming and create new col 'tweet_stemmed'
def word_stemm(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in text]


combine_df['tweet_stemmed'] = combine_df.apply(lambda x: word_stemm(x['tweet_token']), axis = 1)

In [138]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/medic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [139]:
# apply lemmatize and create new col 'tweet_lemmatized'
def word_lemm(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, wordnet.VERB) for word in text]

combine_df['tweet_lemmatized'] = combine_df.apply(lambda x: word_lemm(x['tweet_token']), axis = 1)    

In [140]:
combine_df.head(5)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[when, father, is, dysfunct, and, is, so, selfish, he, drag, hi, kid, into, hi, dysfunct, run]","[when, father, be, dysfunctional, and, be, so, selfish, he, drag, his, kid, into, his, dysfunction, run]"
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,"[thanks, for, lyft, credit, cannot, use, cause, they, donot, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, cannot, use, cause, donot, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, for, lyft, credit, cannot, use, caus, they, donot, offer, wheelchair, van, in, pdx, disapoint, getthank]","[thank, for, lyft, credit, cannot, use, cause, they, donot, offer, wheelchair, vans, in, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, your, majesti]","[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,"[model, love, take, with, all, the, time, in, ur, , ]","[model, love, take, time, ur, , ]","[model, love, take, with, all, the, time, in, ur, , ]","[model, love, take, with, all, the, time, in, ur, , ]"
4,5,0.0,factsguide: society now #motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, now, motiv]","[factsguide, society, now, motivation]"


In [142]:
with open('combine_df.pickle', 'wb') as f:
    pickle.dump(combine_df, f)

In [145]:
# check load data from pickle file
with open('combine_df.pickle', 'rb') as f:
    backup_combine_df = pickle.load(f)

In [146]:
backup_combine_df.head(5)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[when, father, is, dysfunct, and, is, so, selfish, he, drag, hi, kid, into, hi, dysfunct, run]","[when, father, be, dysfunctional, and, be, so, selfish, he, drag, his, kid, into, his, dysfunction, run]"
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,"[thanks, for, lyft, credit, cannot, use, cause, they, donot, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, cannot, use, cause, donot, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, for, lyft, credit, cannot, use, caus, they, donot, offer, wheelchair, van, in, pdx, disapoint, getthank]","[thank, for, lyft, credit, cannot, use, cause, they, donot, offer, wheelchair, vans, in, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, your, majesti]","[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,"[model, love, take, with, all, the, time, in, ur, , ]","[model, love, take, time, ur, , ]","[model, love, take, with, all, the, time, in, ur, , ]","[model, love, take, with, all, the, time, in, ur, , ]"
4,5,0.0,factsguide: society now #motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, now, motiv]","[factsguide, society, now, motivation]"
